# AI Agent for Autonomous Shop Floor Management

**Copyright (c) 2026 Shrikara Kaudambady. All rights reserved.**

This notebook simulates a **Shop Floor Manager AI Agent**. This agent monitors a simplified production environment in real-time, diagnoses potential problems based on production metrics, and takes autonomous actions to keep the factory running optimally. Its goal is to prevent shutdowns by re-ordering inventory before it runs out and scheduling maintenance before a machine fails, all while trying to maintain high production yield.

### 1. Setup and Library Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import sleep

sns.set_theme(style="darkgrid")

### 2. The Simulated Factory Environment
This class represents the 'real world' of our factory. It keeps track of the state of inventory and machines, and has a method to simulate a single production cycle.

In [ ]:
class ShopFloorEnvironment:
    def __init__(self):
        self.state = {
            'inventory': {'screws': 1000, 'casings': 500},
            'machine': {'id': 'M-01', 'wear': 0.1, 'status': 'OPERATIONAL'},
            'yield': 0.99
        }
        self.pending_orders = []

    def run_production_cycle(self):
        # Process pending supply orders
        for order in self.pending_orders:
            order['lead_time'] -= 1
            if order['lead_time'] <= 0:
                self.state['inventory'][order['item']] += order['quantity']
                print(f"LOGISTICS: Supply of {order['quantity']}x {order['item']} has arrived.")
        self.pending_orders = [o for o in self.pending_orders if o['lead_time'] > 0]

        if self.state['machine']['status'] == 'IN_MAINTENANCE':
            self.state['machine']['status'] = 'OPERATIONAL'
            self.state['machine']['wear'] = 0.0
            print("MAINTENANCE: Machine M-01 is back online.")
            return

        # Consume resources
        self.state['inventory']['screws'] -= 50
        self.state['inventory']['casings'] -= 10
        
        # Increase machine wear
        self.state['machine']['wear'] += np.random.uniform(0.01, 0.05)
        
        # Calculate yield based on machine wear and inventory
        yield_degradation = self.state['machine']['wear'] * 0.1
        if self.state['inventory']['screws'] < 0 or self.state['inventory']['casings'] < 0:
            self.state['yield'] = 0.0 # Catastrophic failure
        else:
            self.state['yield'] = 0.99 - yield_degradation + np.random.uniform(-0.01, 0.01)
        self.state['yield'] = max(0, min(1, self.state['yield']))


### 3. The Agent's Toolkit
These are the functions the agent can use to perceive and act upon its environment.

In [ ]:
class AgentTools:
    def __init__(self, env):
        self.env = env
    
    def get_all_metrics(self):
        return self.env.state
    
    def order_supplies(self, item, quantity, lead_time=5):
        print(f"ACTION: Ordering {quantity}x {item}. Lead time is {lead_time} cycles.")
        self.env.pending_orders.append({'item': item, 'quantity': quantity, 'lead_time': lead_time})
        return "Order Placed"
    
    def dispatch_maintenance(self, machine_id):
        if self.env.state['machine']['id'] == machine_id:
            print(f"ACTION: Dispatching maintenance for {machine_id}. Machine will be offline for 1 cycle.")
            self.env.state['machine']['status'] = 'IN_MAINTENANCE'
            return "Maintenance Scheduled"
        return "Machine not found"
    
    def adjust_machine_params(self, machine_id):
        print("ACTION: Tweaking machine parameters to temporarily boost yield.")
        self.env.state['yield'] *= 1.02 # Give a 2% boost
        self.env.state['machine']['wear'] += 0.05 # At the cost of more wear
        return "Parameters Adjusted"

### 4. The Shop Floor Manager AI Agent
This class contains the agent's core logic. It uses its tools to monitor the environment and then follows a set of rules to decide which action to take, if any.

In [ ]:
class ShopFloorManagerAgent:
    def __init__(self, tools):
        self.tools = tools
        self.thresholds = {
            'inventory_low': 200, # Re-order when stock drops below this
            'machine_wear_high': 0.85, # Schedule maintenance above this
            'yield_low': 0.96 # Adjust parameters if yield drops below this
        }
        self.is_ordering = {item: False for item in tools.get_all_metrics()['inventory'].keys()}

    def evaluate_and_act(self):
        print(f"\n--- Agent Decision Cycle ---")
        metrics = self.tools.get_all_metrics()
        
        # 1. Highest priority: Check machine health
        if metrics['machine']['wear'] > self.thresholds['machine_wear_high'] and metrics['machine']['status'] == 'OPERATIONAL':
            print(f"DIAGNOSIS: Machine wear is {metrics['machine']['wear']:.2f}, exceeding threshold of {self.thresholds['machine_wear_high']}.")
            self.tools.dispatch_maintenance(metrics['machine']['id'])
            return
        
        # 2. Next priority: Check inventory levels
        for item, level in metrics['inventory'].items():
            if level < self.thresholds['inventory_low'] and not self.is_ordering[item]:
                print(f"DIAGNOSIS: Inventory for '{item}' is low ({level}).")
                self.tools.order_supplies(item, 5000)
                self.is_ordering[item] = True # Prevent re-ordering while waiting
                return
        # Reset ordering flag if stock is replenished
        for item, level in metrics['inventory'].items():
            if level > self.thresholds['inventory_low']:
                self.is_ordering[item] = False
        
        # 3. Next priority: Check production yield
        if metrics['yield'] < self.thresholds['yield_low']:
            print(f"DIAGNOSIS: Production yield is {metrics['yield']:.2f}, which is below target.")
            self.tools.adjust_machine_params(metrics['machine']['id'])
            return
        
        print("DIAGNOSIS: All metrics are within acceptable parameters. No action needed.")

### 5. Run the Full Simulation
We'll now run the simulation for 100 production cycles. In each cycle, the environment will update, and then the agent will get a chance to evaluate the new state and act.

In [ ]:
env = ShopFloorEnvironment()
tools = AgentTools(env)
agent = ShopFloorManagerAgent(tools)

history = []
N_CYCLES = 100

for cycle in range(N_CYCLES):
    print(f"\n{'='*20} CYCLE {cycle+1}/{N_CYCLES} {'='*20}")
    env.run_production_cycle()
    agent.evaluate_and_act()
    
    current_state = env.state.copy()
    log_entry = {
        'cycle': cycle,
        'screw_inventory': current_state['inventory']['screws'],
        'casing_inventory': current_state['inventory']['casings'],
        'machine_wear': current_state['machine']['wear'],
        'yield': current_state['yield'],
        'machine_status': 1 if current_state['machine']['status'] == 'OPERATIONAL' else 0
    }
    history.append(log_entry)
    # sleep(0.1) # Uncomment for a slower, more readable simulation

history_df = pd.DataFrame(history)

### 6. Analyze Simulation Results
Finally, we'll plot the metrics over time to visualize the agent's performance and see how it managed the environment.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(15, 12), sharex=True)

# Plot Inventory
axes[0].plot(history_df['cycle'], history_df['screw_inventory'], label='Screws')
axes[0].plot(history_df['cycle'], history_df['casing_inventory'], label='Casings')
axes[0].axhline(y=agent.thresholds['inventory_low'], color='r', linestyle='--', label='Low Inventory Threshold')
axes[0].set_title('Inventory Levels Over Time')
axes[0].set_ylabel('Stock Count')
axes[0].legend()

# Plot Machine Wear
axes[1].plot(history_df['cycle'], history_df['machine_wear'], label='Machine Wear', color='orange')
axes[1].axhline(y=agent.thresholds['machine_wear_high'], color='r', linestyle='--', label='Maintenance Threshold')
axes[1].set_title('Machine Wear and Maintenance Cycles')
axes[1].set_ylabel('Wear Level')
# Highlight maintenance periods
maintenance_cycles = history_df[history_df['machine_status'] == 0]['cycle']
for cycle in maintenance_cycles:
    axes[1].axvline(x=cycle, color='g', linestyle='-.', label='Maintenance Action' if 'Maintenance Action' not in [l.get_label() for l in axes[1].get_legend().get_lines()] else "")
axes[1].legend()

# Plot Yield
axes[2].plot(history_df['cycle'], history_df['yield'], label='Production Yield', color='green')
axes[2].axhline(y=agent.thresholds['yield_low'], color='r', linestyle='--', label='Low Yield Threshold')
axes[2].set_title('Production Yield Over Time')
axes[2].set_ylabel('Yield (%)')
axes[2].set_xlabel('Production Cycle')
axes[2].legend()

plt.tight_layout()
plt.show()